### 1. Создание датасета



In [17]:
import pandas as pd
import numpy as np

# Генерация данных с предметами и произвольными оценками
subjects = ['Математика', 'Физика', 'Химия', 'Биология', 'История', 'География']
grades = np.random.randint(0, 100, size=(2000, 6))
lab_grade = np.random.choice(['Удовлетворительно', 'Хорошо', 'Отлично'], size=2000)

# добавление столбца с оценкой по лабораторной работе
data = pd.DataFrame(grades, columns=subjects)
data.insert(6,'Grade', lab_grade)


# Вывод датасета
print(data)


      Математика  Физика  Химия  Биология  История  География  \
0             14      85     51         5       27         98   
1             95       2     55         8       43         60   
2             24      37     37        87       91         15   
3             50      25     37        61       53         71   
4             33      76     63        92       30         39   
...          ...     ...    ...       ...      ...        ...   
1995          19      99     65        50       96         78   
1996          67      53     33        21       88         42   
1997          25      46     42        62       76         29   
1998          50      59     48        18       13         29   
1999          39      31     79        44       98         34   

                  Grade  
0                Хорошо  
1     Удовлетворительно  
2     Удовлетворительно  
3               Отлично  
4     Удовлетворительно  
...                 ...  
1995            Отлично  
1996       

In [19]:
# переводим данные оценко лабораторных работ в котегориальные данные
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(data.Grade)
data.Grade = le.transform(data.Grade)
data.head()

,Математика,Физика,Химия,Биология,История,География,Grade
0,14,85,51,5,27,98,2
1,95,2,55,8,43,60,1
2,24,37,37,87,91,15,1
3,50,25,37,61,53,71,0
4,33,76,63,92,30,39,1


### 2. разбиваем датасет на тестовую и обучающую выборку
### 3. выбираем три лучших признака

In [20]:
# Выводим корреляцию между столбцами и выбираем те 3 столбца значение которых в графе Grade
# Ближе всего к модулю 1.
data.corr()

,Математика,Физика,Химия,Биология,История,География,Grade
Математика,1.000000,0.014556,-0.015673,-0.009033,0.001638,-0.030725,0.046354
Физика,0.014556,1.000000,0.028089,0.013464,0.025332,-0.029082,-0.005501
Химия,-0.015673,0.028089,1.000000,-0.008962,-0.017521,-0.009119,-0.024801
Биология,-0.009033,0.013464,-0.008962,1.000000,0.016409,0.007927,-0.005393
История,0.001638,0.025332,-0.017521,0.016409,1.000000,-0.030918,0.002803
География,-0.030725,-0.029082,-0.009119,0.007927,-0.030918,1.000000,-0.007346
Grade,0.046354,-0.005501,-0.024801,-0.005393,0.002803,-0.007346,1.000000


In [86]:
# Разбиваем датасет на тестовую и обучающую выборки
from sklearn.model_selection import train_test_split
x_train1, x_test1, y_train1, y_test1 = train_test_split(data[['Математика', 'Химия', 'География']], data.Grade, test_size=0.2, random_state=2)


### 4. Проводим обучение модели, результатом которой является сохраненная модель

In [24]:
pip install catboost


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.1 MB/s eta 0:00:00


In [87]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(iterations=500, depth=4, learning_rate=0.1)
model.fit(x_train1, y_train1, silent=True)

### 5. Проводим тестирование можели, результатом которого является число - точность модели на тестовой выборке

In [88]:
accuracy = model.score(x_test1, y_test1)
print(f'точность: {accuracy*100}%')

точность: 32.0%


### 6. Реализуем функцию, которая на вход принимает оценки студентов по 6 предметам и возвращает прогноз оценки.

In [89]:
def predict_grade(math, phis, chem, bio, hist, geo):
  subjects_test = ['Математика','Химия', 'География']
  grades_test = np.array([[math, chem, geo]])


# добавление столбца с оценкой по лабораторной работе
  data_test = pd.DataFrame(grades_test, columns=subjects_test)

  pr = model.predict(data_test)
  result =''
  match pr:
    case 2:
      result='удовлетворительно'
    case 1:
      result ='хорошо'
    case 0:
      result='отлично'

  return result


In [93]:
# Результат работы функции
print(predict_grade(56,2,56,4,5,6))

отлично
